Cleaning the countypres_2000-2024 file 
1. nettoyage du 'bruit', on ne garde que Repliblicans & Democrates
2. on crée des % pour avoir le même poids pour tous les états (500 à LA ce n'est aps la même chose que 500 votes dans un village du texas)
3. reshaping : format long --> format large

In [3]:
import pandas as pd
import numpy as np

# ==========================================
# STEP 1: LOAD DATA
# ==========================================
# Load the raw CSV file using the correct separator
file_path = '/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/raw/election_results/countypres_2000-2024.csv'
df_raw = pd.read_csv(file_path, sep=None, engine='python')

print("✅ Fichier chargé !")
print(f"Colonnes trouvées : {df_raw.columns.tolist()}")

# Display initial shape
print(f"Initial dataset shape: {df_raw.shape}")

# ==========================================
# STEP 2: DATA CLEANING & FILTERING
# ==========================================
# We only want to analyze the two major parties: Democrat and Republican.
# This filters out Green, Libertarian, and other minor parties.
df_clean = df_raw[df_raw['party'].isin(['DEMOCRAT', 'REPUBLICAN'])].copy()
print("✅ Filtrage réussi !")

# Calculate the 'vote_share': the percentage of total votes a candidate received.
# Formula: Candidate Votes / Total Votes in the County
df_clean['vote_share'] = df_clean['candidatevotes'] / df_clean['totalvotes']

# ==========================================
# STEP 3: RESHAPING (PIVOT)
# ==========================================
# Transform the data from "Long Format" (one row per candidate) 
# to "Wide Format" (one row per county per year).
# We want separate columns for Democrat and Republican vote shares.
df_pivot = df_clean.pivot_table(
    index=['year', 'state_po', 'county_name', 'county_fips'], # Unique identifiers for a row
    columns='party',                                          # Values to become new columns
    values='vote_share'                                       # The metric to populate the cells
).reset_index()

# Clean up column names (remove the hierarchy created by pivot_table)
df_pivot.columns.name = None

# Fill NaN values with 0 (in case a party received 0 votes in a county)
df_pivot = df_pivot.fillna(0)

# ==========================================
# STEP 4: EXPORT & VALIDATION
# ==========================================
# Save the processed file for the next steps (Modeling)
output_filename = "votes_cleaned_2000_2024.csv"
df_pivot.to_csv(output_filename, index=False)

print(f"Processing complete. Data saved to '{output_filename}'")
print("\nFirst 5 rows of the cleaned dataset:")
print(df_pivot.head())

✅ Fichier chargé !
Colonnes trouvées : ['year', 'state', 'state_po', 'county_name', 'county_fips', 'office', 'candidate', 'party', 'candidatevotes', 'totalvotes', 'version', 'mode']
Initial dataset shape: (94409, 12)
✅ Filtrage réussi !
Processing complete. Data saved to 'votes_cleaned_2000_2024.csv'

First 5 rows of the cleaned dataset:
   year state_po  county_name  county_fips  DEMOCRAT  REPUBLICAN
0  2000       AK   DISTRICT 1       2001.0  0.192909    0.703275
1  2000       AK  DISTRICT 10       2010.0  0.246961    0.638564
2  2000       AK  DISTRICT 11       2011.0  0.292693    0.567335
3  2000       AK  DISTRICT 12       2012.0  0.269833    0.608604
4  2000       AK  DISTRICT 13       2013.0  0.335012    0.485081


Cleaning file DP03_economic.csv : 
1. on ne garde que 4 variables pour éviter l'Overfitting (chômage, pauvreté, revenus, fonctionnaires)

In [3]:
import pandas as pd
import numpy as np

# ==========================================
# STEP 1: LOAD DATA
# ==========================================
file_path_eco = "/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/raw/socio-economic/DP03_economic.csv"

# Load with header=1 to get the codes
df_eco = pd.read_csv(file_path_eco, header=0)
df_eco = df_eco.iloc[1:] # Drop description row

# ==========================================
# STEP 2: SELECT VARIABLES (CUSTOMIZED)
# ==========================================
# I added 'DP03_0043PE' which is "Government workers" (Percent)
# You can add as many as you want here if you know the code!

columns_mapping = {
    'GEO_ID': 'fips',
    'NAME': 'county_name',
    'DP03_0009PE': 'unemployment_rate',      # Chômage
    'DP03_0062E': 'median_income',           # Revenu
    'DP03_0128PE': 'poverty_rate',           # Pauvreté
    'DP03_0043PE': 'public_workers_pct'      # (BONUS) Fonctionnaires
}

# Keep and rename
df_eco_clean = df_eco[columns_mapping.keys()].rename(columns=columns_mapping).copy()

# ==========================================
# STEP 3: CLEANING & TYPES
# ==========================================
# Convert all value columns to numbers
cols_to_convert = ['unemployment_rate', 'median_income', 'poverty_rate', 'public_workers_pct']

for col in cols_to_convert:
    df_eco_clean[col] = pd.to_numeric(df_eco_clean[col], errors='coerce')

# Clean FIPS (Keep last 5 digits)
df_eco_clean['fips'] = df_eco_clean['fips'].astype(str).str[-5:]

# ==========================================
# STEP 4: SAVE
# ==========================================
df_eco_clean.to_csv("economics_cleaned_2023.csv", index=False)

print("✅ Updated economic file with 4 variables!")
print(df_eco_clean.head())

✅ Updated economic file with 4 variables!
    fips              county_name  unemployment_rate  median_income  \
1  01001  Autauga County, Alabama                2.5        69841.0   
2  01003  Baldwin County, Alabama                3.2        75019.0   
3  01005  Barbour County, Alabama                5.7        44290.0   
4  01007     Bibb County, Alabama               10.0        51215.0   
5  01009   Blount County, Alabama                5.8        61096.0   

   poverty_rate  public_workers_pct  
1          10.7                10.1  
2          10.5                10.1  
3          21.9                 8.6  
4          20.5                 3.4  
5          14.1                 5.9  


/var/folders/q7/2bt1kt6d0xzb95k0cn4jftsr0000gn/T/ipykernel_18391/185637181.py:10: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,144,145,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,276,278,279,280,281,282,283,284,285,286,287,288,289,290,292,293,294,296,297,298

Cleaning the DP02_socio.csv file :


In [5]:
import pandas as pd

# ==========================================
# STEP 1: LOAD SOCIAL DATA (DP02)
# ==========================================
# ⚠️ Update the path below with your own "Copy Path" for DP02
file_path_socio = "/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/raw/socio-economic/DP02_socio.csv"

df_socio = pd.read_csv(file_path_socio, header=0)
df_socio = df_socio.iloc[1:] # Drop description row

print(f"✅ Social Data Loaded! Shape: {df_socio.shape}")

# ==========================================
# STEP 2: SELECT EDUCATION DATA
# ==========================================
# Key Variable:
# - DP02_0068PE: Percent of people (25+) with a Bachelor's degree or higher

columns_mapping_socio = {
    'GEO_ID': 'fips',
    'DP02_0068PE': 'education_bachelors_pct'
}

df_socio_clean = df_socio[columns_mapping_socio.keys()].rename(columns=columns_mapping_socio).copy()

# ==========================================
# STEP 3: CLEANING
# ==========================================
# Convert to numbers
df_socio_clean['education_bachelors_pct'] = pd.to_numeric(df_socio_clean['education_bachelors_pct'], errors='coerce')

# Clean FIPS code (last 5 characters)
df_socio_clean['fips'] = df_socio_clean['fips'].astype(str).str[-5:]

# ==========================================
# STEP 4: SAVE
# ==========================================
output_filename_socio = "education_cleaned_2023.csv"
df_socio_clean.to_csv(output_filename_socio, index=False)

print(f"🎉 Process Complete! File saved: {output_filename_socio}")
print(df_socio_clean.head())

✅ Social Data Loaded! Shape: (3222, 619)
🎉 Process Complete! File saved: education_cleaned_2023.csv
    fips  education_bachelors_pct
1  01001                     28.3
2  01003                     32.8
3  01005                     11.5
4  01007                     11.5
5  01009                     15.6


/var/folders/q7/2bt1kt6d0xzb95k0cn4jftsr0000gn/T/ipykernel_18391/2107103794.py:9: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,250,251,252,253,254,2

Cleaning the file DP05_demo.csv :
1. on garde la variable age 
2. on garde les variables relatives à l'origine éthnique

In [6]:
import pandas as pd

# ==========================================
# STEP 1: LOAD DEMOGRAPHIC DATA (DP05)
# ==========================================
# ⚠️ Remplacez le chemin par celui de votre VRAI fichier DP05 sur votre ordinateur
file_path_demo = "/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/raw/socio-economic/DP05_demo.csv"

df_demo = pd.read_csv(file_path_demo, header=0)
df_demo = df_demo.iloc[1:] # On enlève la ligne de description

print(f"✅ Fichier Démo chargé ! Taille : {df_demo.shape}")

# ==========================================
# STEP 2: SELECT VARIABLES
# ==========================================
# Voici les variables clés pour la politique US :
# - DP05_0018E : Age Médian (Les jeunes votent plus à gauche, les âgés à droite)
# - DP05_0037PE : % Blancs (White alone)
# - DP05_0038PE : % Noirs (Black or African American alone)
# - DP05_0071PE : % Hispaniques (Hispanic or Latino)

columns_mapping_demo = {
    'GEO_ID': 'fips',
    'DP05_0018E': 'median_age',
    'DP05_0037PE': 'white_pct',
    'DP05_0038PE': 'black_pct',
    'DP05_0071PE': 'hispanic_pct'
}

# On sélectionne et renomme
df_demo_clean = df_demo[columns_mapping_demo.keys()].rename(columns=columns_mapping_demo).copy()

# ==========================================
# STEP 3: CLEANING
# ==========================================
# Conversion en nombres
cols_to_convert = ['median_age', 'white_pct', 'black_pct', 'hispanic_pct']

for col in cols_to_convert:
    df_demo_clean[col] = pd.to_numeric(df_demo_clean[col], errors='coerce')

# Nettoyage du code FIPS (les 5 derniers chiffres)
df_demo_clean['fips'] = df_demo_clean['fips'].astype(str).str[-5:]

# ==========================================
# STEP 4: SAVE
# ==========================================
output_filename_demo = "demographics_cleaned_2023.csv"
df_demo_clean.to_csv(output_filename_demo, index=False)

print(f"🎉 Terminé ! Fichier sauvegardé : {output_filename_demo}")
print(df_demo_clean.head())

✅ Fichier Démo chargé ! Taille : (3222, 379)
🎉 Terminé ! Fichier sauvegardé : demographics_cleaned_2023.csv
    fips  median_age  white_pct  black_pct  hispanic_pct
1  01001        39.2       73.6       20.0           0.8
2  01003        43.7       82.8        8.0           1.6
3  01005        40.7       44.0       46.9           0.9
4  01007        41.3       75.1       20.7           1.9
5  01009        40.9       89.5        1.3           1.7


/var/folders/q7/2bt1kt6d0xzb95k0cn4jftsr0000gn/T/ipykernel_18391/1783815027.py:9: DtypeWarning: Columns (2,4,6,8,10,12,13,14,15,16,18,20,22,24,26,27,28,29,30,32,33,34,35,36,37,38,40,41,42,44,45,46,47,48,50,52,54,56,58,60,62,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,138,139,140,141,142,143,144,145,146,147,148,149,150,152,154,155,156,157,158,159,160,161,162,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,192,194,198,200,201,202,203,204,206,208,210,212,214,215,216,217,218,220,221,222,223,226,228,229,230,232,233,234,235,236,238,240,242,246,248,250,254,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,3

Master table (Merge to obtain our structured database)

In [10]:
import pandas as pd

# ==========================================
# STEP 1: LOAD (Colle tes chemins ici)
# ==========================================
path_votes = "/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/processed/votes_cleaned_2000_2024.csv"
path_eco = "/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/processed/economics_cleaned_2023.csv"
path_edu = "/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/processed/education_cleaned_2023.csv"
path_demo = "/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/processed/demographics_cleaned_2023.csv"

# On charge tout
df_votes = pd.read_csv(path_votes)
df_eco = pd.read_csv(path_eco)
df_edu = pd.read_csv(path_edu)
df_demo = pd.read_csv(path_demo)

# Standardisation du nom de colonne
df_votes = df_votes.rename(columns={'county_fips': 'fips'})

# ==========================================
# 🚨 STEP 1.5: LA LESSIVEUSE À FIPS (CRUCIAL)
# ==========================================
def clean_fips(df):
    # 1. On force en numérique pour gérer les "1001.0"
    df['fips'] = pd.to_numeric(df['fips'], errors='coerce')
    # 2. On enlève les lignes où le FIPS est vide/cassé
    df = df.dropna(subset=['fips'])
    # 3. On transforme en Entier (int) pour tuer le ".0" -> 1001
    df['fips'] = df['fips'].astype(int)
    # 4. On transforme en Texte et on ajoute les zéros -> "01001"
    df['fips'] = df['fips'].astype(str).str.zfill(5)
    return df

print("--- Nettoyage des FIPS en cours ---")
df_votes = clean_fips(df_votes)
df_eco   = clean_fips(df_eco)
df_edu   = clean_fips(df_edu)
df_demo  = clean_fips(df_demo)

print(f"✅ Exemple FIPS Votes (Doit être 01001) : '{df_votes['fips'].iloc[0]}'")
print(f"✅ Exemple FIPS Eco   (Doit être 01001) : '{df_eco['fips'].iloc[0]}'")

# Si les deux affichent '01001' (sans .0), la suite marchera !

# ==========================================
# STEP 2: FUSION
# ==========================================
print("\nFusion...")
df_master = pd.merge(df_votes, df_eco, on='fips', how='inner') # On tente 'inner' pour voir ce qui matche vraiment
df_master = pd.merge(df_master, df_edu, on='fips', how='left')
df_master = pd.merge(df_master, df_demo, on='fips', how='left')

# ==========================================
# STEP 3: NETTOYAGE COLONNES
# ==========================================
cols_to_drop = [c for c in df_master.columns if c.endswith('_y') or 'county_name_eco' in c]
df_master = df_master.drop(columns=cols_to_drop)
df_master.columns = df_master.columns.str.replace('_x', '')

# ==========================================
# STEP 4: RÉSULTAT
# ==========================================
print(f"\n📊 Taille finale de la Master Table : {len(df_master)} lignes.")
if len(df_master) > 0:
    df_master.to_csv("master_table_elections.csv", index=False)
    print("🎉 SAUVEGARDÉ ! Le fichier est plein.")
    print(df_master.head())
else:
    print("❌ TOUJOURS VIDE... Le problème est plus profond.")

--- Nettoyage des FIPS en cours ---
✅ Exemple FIPS Votes (Doit être 01001) : '02001'
✅ Exemple FIPS Eco   (Doit être 01001) : '01001'

Fusion...

📊 Taille finale de la Master Table : 21742 lignes.
🎉 SAUVEGARDÉ ! Le fichier est plein.
   year state_po  county_name   fips  DEMOCRAT  REPUBLICAN  unemployment_rate  \
0  2000       AK  DISTRICT 13  02013  0.335012    0.485081                3.8   
1  2000       AK  DISTRICT 16  02016  0.420277    0.422625                4.1   
2  2000       AK  DISTRICT 20  02020  0.324429    0.523912                4.6   
3  2000       AL      AUTAUGA  01001  0.287192    0.696943                2.5   
4  2000       AL      BALDWIN  01003  0.247822    0.723654                3.2   

   median_income  poverty_rate  public_workers_pct  education_bachelors_pct  \
0        72692.0          12.4                 1.9                     18.1   
1       107344.0           9.2                 4.4                     17.5   
2        98152.0           9.3            